#### Student Name: Mai Ngo
#### Course Name and Number: DSC 430 Python Programming
#### Assignment Name and Number: Assignment0602_SurfCDM
#### Date: 2/15/2023
#### Honor Statement: “I have not given or received any unauthorized assistance on this assignment."

#### Assumption

I put a lot of commands '#' after each code, because I'm learning while doing this homework. Make note to understand what each line means specifically. \
Minor check to make sure I'm doing it right:

    *URL: ' https://depaulmagazine.com/2022/12/08/professional-partners/" has leading whitespace. 
    *NOTE: Just CDM homepage: 187 count '<a' tag | 173 count URL links | 9 '<a' tags in footer | 5 '<a' tags don't start with 'http'

#### Show how you extended HTMLParser.

I coded handle_starttag() to get all hyperlinks correctly. \
For handle_data, I converted all data to lower case.

#### Show which methods you overwrote.

I created new def cleanWord() to remove stop word, and remove special characters.

#### Show how you restricted your search to webpages at CDM.

I overwrote analyze()- to only get hyper links start with base link 'https://www.cdm.depaul.edu/'. 

In [1]:
from urllib.request import urlopen, urljoin, urlparse, Request
from html.parser import HTMLParser
import collections
from collections import Counter
#Import libraries

In [109]:
class Collector(HTMLParser):
    """Collects hyperlink URLs into a list"""

    ## (*) For our purpose, we want to ignore several tags that prevents
    ## extraction of clean text.  We define them as a class variable.
    ignore_tags = ['script', 'noscript', 'input', 'meta', 'title', 'style', 'form']
    tag = ''
    def __init__(self, url):
        """ initializes parser, the url, and a list """
        HTMLParser.__init__(self)
        self.url = url
        self.links = []
        self.data = []

    def handle_starttag(self, tag, attrs):
        """ collects hyperlink URLs in their absolute format """      
        self.tag = tag
        #Assign tag to an instance variable.
        if self.tag == 'a':
        #<a> tag defines a hyperlink
        
            for attr in attrs:
            #Iterate over each piece of HTML. 
                if attr[0] == 'href':
                #Specifies a URL.  
                
                    absolute = urljoin(self.url, attr[1].strip())
                    #Assign absolute variable: Construct absolute URL, join base URL and path attr[1]. 
                    #Using strip() because ' https://depaulmagazine.com/2022/12/08/professional-partners/' has leading whitespace 
                    
                    if absolute[:4] == 'http': # collect HTTP URLs
                        self.links.append(absolute)
                   
    def handle_data(self, data):
        if self.tag not in self.ignore_tags:
            for word in data.strip().split():
                gen_word = word.lower()
                self.data.append(gen_word)

    def getLinks(self):
        """ returns hyperlinks URLs in their absolute format """
        return self.links

    def getData(self):
        """ returns the data (accumulated in the instance variable) """
        return self.data

In [99]:
def analyze(url):
    '''Returns list of all http links in url, in absolute format.'''
    
    print('\n\nVisiting', url)           
    #Print the initial visiting url - CDM Website

    user_agent = "Mozilla/5.0 (Windows NT 6.1; Win64; x64)"
    #Assign crawler agent to avoid HTTP Error 418
    request = Request(url)
    #Send HTTP request using the URL
    
    request.add_header("User-Agent",user_agent)
    #Tells the web server what kind of operating system and browser: (Mozilla/5.0)
    content = urlopen(request).read().decode()
    #Makes a HTTP transaction to retrieve the associated resource.
    
    collector = Collector(url)
    collector.feed(content)
    #Feed a string containing HTML, processes it
    urls = collector.getLinks()
    
    url_s = []
    #Collect URL within CDM.
    for url in urls:
    #Iterate over each url from getLinks()
        if url[:27] == 'https://www.cdm.depaul.edu/':
        #If the hyperlink has the same base link.
            url_s.append(url)   
            #Then append.
        else: pass
    
    content = collector.getData()
    #Retrieve content, have converted to all lower case. 
    cleandata = cleanWord(content)
    #Clean data once again, special chars, stop words.
    
    word_count = frequency(cleandata)
    #Give frequency of each word in content.
    
    return word_count

In [100]:
def cleanWord(content):
    '''To remove stop word, and remove special characters.'''
   
    filename = "C:/Users/maimu/OneDrive/Documents/DePaul/DSC 430//M6_stopwords.txt"
    stopWords = []
    with open(filename) as infile:
    #Open stop word file and convert into a list of word.
        for word in infile: 
            word = word.strip() 
            stopWords.append(word)
    
    cleanWordlst = []
    for word in content:
    #Iterate through each word.
    
        symbols = "!@#$%^&*()_-+={[}]|\;:\"<>?/., "
        #Potential symbols in content.
        for i in range(len(symbols)):
        #Iterate over each symbol.
            word = word.replace(symbols[i], '')
            #Replace special character with nothing.
            
        if len(word) > 1:
        #If word length is greater than 1, which meanns not speical character. 
            cleanWordlst.append(word)
            #Append the word to final clean content.
        
    cleanWordlst2 = []
    #Finbal clean content.
    for word2 in cleanWordlst:
    #Iterate through wordlist after remove all special characters. 
    
        if word2 not in stopWords:
        #If the word is not in stop words list.
            cleanWordlst2.append(word2)
                
    return cleanWordlst2

In [93]:
def frequency(content):
    '''Return count of each word.'''
    #sentence is a list of words.
    
    word_count = Counter(content)
    #Return each word's count.
    
    wordCount = {}
    #Assign wordCount variable to store count, as dictionary.
    
    for wordlst in content:
    #Iterate over each word-component in 'sentence' list.
    #wordlst variable is a string.
    
        words = wordlst.strip().split()
        #Assign variable 'words' splits 'wordlst' string into a list 'words'
        for word in words:
            #Iterate over each word in 'words' list.
            gen_word = word.lower()
            if word in wordCount:
            #Check if the word is in already in wordCount dictionary.
                wordCount[word] += 1
                #If so, add one more count to the respective word.
                
            else:
            #If the word is not in the wordCount dictionary.
                wordCount[word] = 1
                #Adding new word with count as 1.
    
    return word_count
    #return word_count.

In [101]:
visited = set() 
#Initialize an empty set 'visited'. Each stored link will be unique. 

def crawl(url):
    ''' Recursive web crawler that calls analyze() on every visited web page.'''
    
    global visited
    #Declare 'visited' throughout the programme. Links already in there will not be visited again.
    visited.add(url)
    #Add url link to 'visited' set after crawl.


    links = analyze(url)
    #Assign variable 'links', returns a list of URL hyperlinks in visited web page.

    for link in links:
    #Iterate over each URL link in 'links' list
        if link not in visited:
        #If link has not been visited yet. 
        
            try: crawl(link)
            #Execute using def crawl(), which using def analyze()
            #Recursively continue crawl from every link in links.
            
            except: pass
            #In case link is already visited. Pass to next link.
            
        else: pass

In [102]:
def printOutput(wordCount):
    '''Test and write palindrome dates to a file.'''
    #Given a list of reversed year numbers (4 digits), and a list of date (2 digits). 
    
    outfile = open('HW0602.txt', 'w', encoding = "utf-8")
    #Create/Open a text file named 'HW0602' to write only.
    outfile.write('The most 50 common words on the CDM website: \n')
    #Give header
        
    common = wordCount.most_common(50)
    for word1, count1 in common:
            outfile.write('{:20} {:5} \n'.format(word1, count1))
            
    outfile.close()
    #Close the file at the end.

In [110]:
def main():
    url = 'https://www.cdm.depaul.edu/'
    word_count = analyze(url)
    wordCount = frequency(word_count)
    printOutput(wordCount)

In [111]:
main()



Visiting https://www.cdm.depaul.edu/
